In [1]:
from datetime import datetime
import logging
from typing import List
import pandas as pd
import datetime
import os

from camply.containers import AvailableCampsite, SearchWindow
from camply.search import SearchRecreationDotGov, SearchReserveCalifornia
from camply.config import EmailConfig

In [2]:
AreaCSV = pd.read_csv('../data/rec_area_names.csv')
print(AreaCSV.head())

   RecAreaID                               RecAreaName
0          4                                 WestWorld
1          6                             Lake Pleasant
2          9                      Shadow Mountain Lake
3         10  White Mountains National Recreation Area
4         11                               Folsom Lake


In [3]:
user_df = pd.read_csv('../data/UserInfo.csv')
print(user_df.head())

  Date of Query         Name                       Rec Area  Weeks in Advance  \
0    11/30/2024  Chad Breece        Pinnacles National Park                 3   
1    11/30/2024  Chad Breece  Point Reyes National Seashore                 3   

   Number of Nights  Weekends Only                Email  
0                 1              1  cmebreece@gmail.com  
1                 1              1  cmebreece@gmail.com  


In [4]:

uniq_users = user_df['Name'].unique()
for user in uniq_users:
    ### NOTE: Need a seperate thread for each user
    #get email address for user
    to_email_address = user_df[user_df['Name'] == user]['Email'].unique()
    #get rec area ids for user
    user_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
    uniq_recs = [int(AreaCSV[AreaCSV['RecAreaName'] == rec]['RecAreaID'].values[0]) for rec in user_recs]

    #get number of weeks of advanced notice
    adv_notice_weeks = int(user_df[user_df['Name'] == user]['Weeks in Advance'].unique()[0])
    #calculate end date
    adv_notice_days = adv_notice_weeks * 7
    u = datetime.date.today()
    d = datetime.timedelta(days=adv_notice_days)
    start = u
    end = u + d
    print(start, end)
    date_range = SearchWindow(start_date = start, end_date = end)
    weekend_only_bool = bool(user_df[user_df['Name'] == user]['Weekends Only'].unique()[0])
    num_nights = int(user_df[user_df['Name'] == user]['Number of Nights'].unique()[0])
    
    # Search for campsites for user
    camping_finder = SearchRecreationDotGov(search_window=date_range,
                                            recreation_area=uniq_recs,  # Glacier Ntl Park
                                            weekends_only=weekend_only_bool,
                                            nights=num_nights,
                                            equipment=[("Tent", None)],
                                            offline_search=True,
                                            offline_search_path='./offline_searches/user.json'
                                            )
    # Get matching campsites
    matches = camping_finder.get_matching_campsites(log=True, verbose=True,
                            continuous=True,
                            polling_interval=5,
                            search_forever=True,
                            notify_first_try=False,
                            notification_provider='email'
                            )


2024-12-02 2024-12-23


SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. For more\n5.7.9 information, go to\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired 98e67ed59e1d1-2ee96627b05sm4457596a91.15 - gsmtp')

In [89]:
matches

[AvailableCampsite(campsite_id=92460, booking_date=datetime.datetime(2024, 12, 14, 0, 0), booking_end_date=datetime.datetime(2024, 12, 15, 0, 0), booking_nights=1, campsite_site_name='10', campsite_loop_name='A', campsite_type='STANDARD NONELECTRIC', campsite_occupancy=(0, 6), campsite_use_type='Overnight', availability_status='Available', recreation_area='Pinnacles National Park, CA', recreation_area_id=2893, facility_name='Pinnacles Campground', facility_id=234015, booking_url='https://www.recreation.gov/camping/campsites/92460', location=CampsiteLocation(latitude=np.float64(36.491522264), longitude=np.float64(-121.146270064)), permitted_equipment=[RecDotGovEquipment(equipment_name='Trailer', max_length=29.0), RecDotGovEquipment(equipment_name='Tent', max_length=29.0), RecDotGovEquipment(equipment_name='RV', max_length=29.0)], campsite_attributes=[RecDotGovAttribute(attribute_category='site_details', attribute_id=52, attribute_name='Max Num of People', attribute_value='6'), RecDotGov

In [6]:
from smtplib import SMTP_SSL

In [7]:
email_server = SMTP_SSL("smtp.gmail.com", 465)
email_server.ehlo()
email_server.login(user='camp.alert.io@gmail.com', password='viqmdtvschrwlcqz')

SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. For more\n5.7.9 information, go to\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired 98e67ed59e1d1-2ef02a6685bsm219184a91.29 - gsmtp')